In [1]:
from contrans import contrans

In [2]:
import numpy as np
import pandas as pd
import json

In [3]:
ct = contrans()

In [4]:
useragent = ct.get_useragent()
useragent

'python-requests/2.32.3'

In [5]:
headers = ct.make_headers(useragent)

In [6]:
ct.get_bioguideIDs()

,bioguideId,district,name,partyName,state,updateDate,url,terms.item,depiction.attribution,depiction.imageUrl
0,M001229,10.0,"McIver, LaMonica",Democratic,New Jersey,2024-09-24T20:50:13Z,https://api.congress.gov/v3/member/M001229?for...,"[{'chamber': 'House of Representatives', 'star...",NaN,NaN
1,E000236,NaN,"Evans, Daniel J.",Republican,Washington,2024-09-23T14:08:49Z,https://api.congress.gov/v3/member/E000236?for...,"[{'chamber': 'Senate', 'endYear': 1989, 'start...","<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/e000236_20...
2,S000068,NaN,"Sasser, Jim",Democratic,Tennessee,2024-09-12T19:33:39Z,https://api.congress.gov/v3/member/S000068?for...,"[{'chamber': 'Senate', 'endYear': 1995, 'start...","<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/s000068_20...
3,H001097,NaN,"Helmy, George S.",Democratic,New Jersey,2024-09-12T12:42:19Z,https://api.congress.gov/v3/member/H001097?for...,"[{'chamber': 'Senate', 'startYear': 2024}]",NaN,NaN
4,R000599,25.0,"Ruiz, Raul",Democratic,California,2024-09-11T16:29:00Z,https://api.congress.gov/v3/member/R000599?for...,"[{'chamber': 'House of Representatives', 'star...",Image courtesy of the Member,https://www.congress.gov/img/member/66e1aec832...
...,...,...,...,...,...,...,...,...,...,...
2519,F000229,6.0,"Flynt, John J., Jr.",Democratic,Georgia,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000229?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000229_20...
2520,F000218,7.0,"Flowers, Walter",Democratic,Alabama,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000218?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000218_20...
2521,F000215,1.0,"Florio, James J.",Democratic,New Jersey,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000215?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000215_20...
2522,F000209,11.0,"Flood, Daniel J.",Democratic,Pennsylvania,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000209?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000209_20...


In [14]:
house

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.2
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
486659,118,House,1118,31101,6,100.0
486660,118,House,1118,31102,9,99.6
486661,118,House,1118,39301,6,100.0
486662,118,House,1118,39307,6,100.0


In [16]:
house = house[["rollnumber", "icpsr", "cast_code"]]

In [17]:
house_mat = pd.merge(house, house, on="rollnumber")

In [19]:
len(house_mat)

211856536

In [20]:
house_mat["agree"] = house_mat["cast_code_x"] == house_mat["cast_code_y"]

In [22]:
house_mat = house_mat.groupby(["icpsr_x", "icpsr_y"]).agg({"agree": "mean"})

In [23]:
house_mat = house_mat.reset_index()

In [24]:
house_mat

,icpsr_x,icpsr_y,agree
0,14854,14854,1.000000
1,14854,14863,0.830948
2,14854,14873,0.386404
3,14854,15029,0.412343
4,14854,15433,0.366726
...,...,...,...
200607,91980,31102,0.731664
200608,91980,31103,0.190805
200609,91980,39301,0.236136
200610,91980,39307,0.239714


In [25]:
ideo = ct.get_ideology()

In [26]:
members = ideo[["bioname", "icpsr", "party_code"]]

In [27]:
members

,bioname,icpsr,party_code
0,"ROGERS, Mike Dennis",20301,200
1,"SEWELL, Terri",21102,100
2,"PALMER, Gary James",21500,200
3,"CARL, Jerry L.",22108,200
4,"MOORE, Barry",22140,200
...,...,...,...
443,"STEIL, Bryan",21970,200
444,"TIFFANY, Thomas P.",21989,200
445,"FITZGERALD, Scott",22115,200
446,"VAN ORDEN, Derrick",22370,200
